Feb 6 2019 class

In [45]:
import math
import numpy as np
import tellurium as te
# this is our calculation to check our numerical calculations
# 1 liter = 1 dm^3
area = 1.0e-10; # dm^2
permeability = 1.0e-8; # dm s-1
km = 5.5e-3; # M
S1 = 10e-3; # M
kcat = 6.42e2; # s-1
etot = 1.0e-6; # M
vol_cell = 1.0e-15; # L or dm^3
a = area * permeability;
b = km*area*permeability-area*permeability*S1+kcat*etot*vol_cell;
c = -1*km*area*permeability*S1;
S3_ss=(-b+(b**2-4*a*c)**0.5)/(2*a);
print(a,b,c)
print('S3_ss',S3_ss)


1e-18 6.375000000000001e-19 -5.5e-23
S3_ss 8.626283721240081e-05


In [46]:
J_ss1=area*permeability*(S1-S3_ss);
print('J_ss1',J_ss1)
J_ss2=(kcat*etot*vol_cell*S3_ss)/(km+S3_ss);
print('J_ss2',J_ss2)

J_ss1 9.9137371627876e-21
J_ss2 9.91373716278571e-21


In [47]:
model LacPath
    compartment cytoplasm;
    cytoplasm = 1.0;
    compartment environment;
    environment = 1000.0;
    species lactose, glucgalac;
    const species ex_lactose;
    unit M = mole / liter
    unit inv_sec = 1. / seconds
    unit um = 10.e-6 meters
    unit dm = 10.e-2 meters
    unit vol_conv = 1 liter / 1 dm
    
    lactose = 0.0 M; # M; free parameter
    glucgalac = 0.0 M;
    ex_lactose = 0.1 M;

    lactose in cytoplasm; glucgalac in cytoplasm;
    ex_lactose in environment
    
    J34: lactose -> glucgalac; kcat*Etot*lactose/(Km+lactose);

    kcat = 6.42e2 inv_sec; #s^-1; from Juers et al (2012) 
    Etot = 1.e-6 M; # M; free parameter
    Km = 5.5e-3 M; # M; from BRENDA
    
    J13: ex_lactose => lactose; area*permeability*(ex_lactose-lactose);
    #J31: lactose => ex_lactose; area*permeability*(lactose-ex_lactose)*conversion;
    
    area = 1e-10 dm^2;
    permeability = 1e-8 dm * 1 inv_sec;
    conversion = 1 vol_conv;
end

RuntimeError: Errors encountered when trying to load model into Antimony:
Error in model string, line 33:  3 SBML error(s) when creating module 'LacPath'.  libAntimony tries to catch these errors before libSBML complains, but sometimes cannot.  Error message(s) from libSBML:

Outside of a <functionDefinition>, if a <ci> element is not the first element within a MathML <apply>, then the <ci>'s value can only be chosen from the set of identifiers of (in L2V1) <species>, <compartment>, or <parameter> objects; (in L2V2-L2V5), <species>, <compartment>, <parameter> or <reaction> objects; (in L3V1) <species>, <compartment>, <parameter>, <reaction> or <speciesReference> objects and (in L3V2) <species>, <compartment>, <parameter>, <reaction>, <speciesReference> objects or L3 package objects with defined mathematical meaning. 
Reference: L3V1 Section 3.4.3
 The formula 'area' in the math element of the <kineticLaw> uses 'area' that is not the id of a species/compartment/parameter/reaction/specie

In [48]:
LacPath.reset()
LacPath.simulate(0., 100., 1000) #'S1','J1'
LacPath.plot()

values = LacPath.getSteadyStateValues()
selections = LacPath.steadyStateSelections
print(values)
print(selections)

[ 0.  0.]
['[lactose]', '[glucgalac]']
